<a href="https://colab.research.google.com/github/Cpillar/-liunx-web-/blob/main/Hemo_finder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

df = pd.read_csv('train_feature.csv')
df_test = pd.read_csv('test_feature.csv')
df = df.sample(frac=1).reset_index(drop=True)
df_test = df_test.sample(frac=1).reset_index(drop=True)
X_train = df.iloc[:, 2:]
y_train = df.iloc[:, 0].str.split('_', expand=True)[1].astype(int)
X_test = df_test.iloc[:, 2:]
y_test = df_test.iloc[:, 0].str.split('_', expand=True)[1].astype(int)

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, matthews_corrcoef, precision_score, recall_score
import os
from subprocess import call
import pandas as pd
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, matthews_corrcoef, precision_score, recall_score
import xgboost as xgb
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
import joblib
classifiers = {
    'LR': LogisticRegression(max_iter=20),
    'XGB': XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
    'DT': DecisionTreeClassifier(),
    'KNN': KNeighborsClassifier(),
    'RF': RandomForestClassifier(),
    'ANN': MLPClassifier(hidden_layer_sizes=(128, 64, 32, 16), max_iter=20)
}


In [12]:
# Train classifiers
for name, clf in classifiers.items():
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(f'{name} Confusion Matrix:\n', confusion_matrix(y_test, y_pred))
    print(f'{name} Accuracy: {accuracy_score(y_test, y_pred)}')
    print(f'{name} MCC: {matthews_corrcoef(y_test, y_pred)}')
    print(f'{name} Precision: {precision_score(y_test, y_pred, average="weighted")}')
    print(f'{name} Recall: {recall_score(y_test, y_pred, average="weighted")}')

/Users/wendy/opt/anaconda3/envs/sklearn-env/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LR Confusion Matrix:
 [[195  95]
 [ 80 249]]
LR Accuracy: 0.7172859450726979
LR MCC: 0.4310864916008123
LR Precision: 0.7169286033874456
LR Recall: 0.7172859450726979


/Users/wendy/opt/anaconda3/envs/sklearn-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [15:33:47] WARNING: /var/folders/c_/qfmhj66j0tn016nkx_th4hxm0000gp/T/abs_b6qk1lz_ug/croot/xgboost-split_1724073748391/work/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGB Confusion Matrix:
 [[254  36]
 [ 68 261]]
XGB Accuracy: 0.8319870759289176
XGB MCC: 0.6683909872777538
XGB Precision: 0.8366380873931623
XGB Recall: 0.8319870759289176
DT Confusion Matrix:
 [[222  68]
 [ 98 231]]
DT Accuracy: 0.7318255250403877
DT MCC: 0.466984609745871
DT Precision: 0.7356458118337377
DT Recall: 0.7318255250403877
KNN Confusion Matrix:
 [[206  84]
 [ 66 263]]
KNN Accuracy: 0.7576736672051696
KNN MCC: 0.5124999803933925
KNN Precision: 0.7576570301201074
KNN Recall: 0.7576736672051696
RF Confusion Matrix:
 [[242  48]
 [ 63 266]]
RF Accuracy: 0.8206785137318255
RF MCC: 0.6417837510788142
RF Precision: 0.8219795913169056
RF Recall: 0.8206785137318255
ANN Confusion Matrix:
 [[223  67]
 [ 55 274]]
ANN Accuracy: 0.802907915993538
ANN MCC: 0.6037316748726239
ANN Precision: 0.8028815356083563
ANN Recall: 0.802907915993538


/Users/wendy/opt/anaconda3/envs/sklearn-env/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


In [13]:
# Ensemble Voting Classifier
voting_clf = VotingClassifier(estimators=[
    ('lr', classifiers['LR']),
    ('xgb', classifiers['XGB']),
    # ('dt', classifiers['DT']),
    ('knn', classifiers['KNN']),
    ('rf', classifiers['RF']),
    ('ann', classifiers['ANN'])
], voting='soft')

voting_clf.fit(X_train, y_train)
y_pred_voting = voting_clf.predict(X_test)
print('Voting Classifier Confusion Matrix:\n', confusion_matrix(y_test, y_pred_voting))
print('Voting Classifier Accuracy: ', accuracy_score(y_test, y_pred_voting))
print('Voting Classifier MCC: ', matthews_corrcoef(y_test, y_pred_voting))
print('Voting Classifier Precision: ', precision_score(y_test, y_pred_voting, average="weighted"))
print('Voting Classifier Recall: ', recall_score(y_test, y_pred_voting, average="weighted"))


/Users/wendy/opt/anaconda3/envs/sklearn-env/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/wendy/opt/anaconda3/envs/sklearn-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [15:33:55] WARNING: /var/folders/c_/qfmhj66j0tn016nkx_th4hxm0000gp/T/abs_b6qk1lz_ug/croot/xgboost-split_1724073748391/work/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/wendy/opt/anaconda3/envs/sklearn-env/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:

Voting Classifier Confusion Matrix:
 [[239  51]
 [ 51 278]]
Voting Classifier Accuracy:  0.8352180936995154
Voting Classifier MCC:  0.6691227334660937
Voting Classifier Precision:  0.8352180936995154
Voting Classifier Recall:  0.8352180936995154


In [ ]:
joblib.dump(voting_clf,"voting_classifier_model_s1.pkl")

['voting_classifier_model_s1.pkl']